# setup

In [1]:
import torch

In [2]:
import os

if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('..')

In [3]:
from dataloader import Cifar100
from runners.pretrain import Runner
from utils import val_split
from hparams import EPOCHS, BATCH_SIZE, NUM_WORKERS

In [4]:
EPOCHS = 3

# init

In [5]:
reflexo = 'resnet18'
origem = 'resnet34'
model_label = reflexo + origem

model = torch.hub.load('pytorch/vision:v0.10.0', reflexo, pretrained = False)
model = list(model.children())[:-1]
model = torch.nn.Sequential(*model)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Using cache found in /home/josegfer/.cache/torch/hub/pytorch_vision_v0.10.0


In [9]:
trn_ds = Cifar100(split = 'train', embedding_path = 'output/{}/H_trn.pt'.format(origem))
# trn_loader = torch.utils.data.DataLoader(trn_ds, batch_size = BATCH_SIZE,
#                                           shuffle = True, num_workers = NUM_WORKERS)
sampler = val_split(N = trn_ds.__len__())
trn_loader = torch.utils.data.DataLoader(trn_ds, batch_size = BATCH_SIZE, 
                                         sampler = sampler['trn_sampler'], num_workers = NUM_WORKERS)
val_loader = torch.utils.data.DataLoader(trn_ds, batch_size = BATCH_SIZE, 
                                         sampler = sampler['val_sampler'], num_workers = NUM_WORKERS)

# tst_ds = Cifar100(split = 'val', embedding_path = 'output/{}/H_tst.pt'.format(origem))
# tst_loader = torch.utils.data.DataLoader(tst_ds, batch_size = BATCH_SIZE,
#                                          shuffle = False, num_workers = NUM_WORKERS)

In [10]:
runner = Runner(device, model, model_label)

# train

In [11]:
runner.train(EPOCHS, trn_loader, val_loader)

-- epoch 0


  0%|          | 0/704 [00:00<?, ?it/s]/home/josegfer/miniconda3/envs/mirror/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 79/79 [00:06<00:00, 12.03it/s]


new checkpoint with val loss: 0.1852433491922632
-- epoch 1


100%|██████████| 79/79 [00:06<00:00, 13.10it/s]


new checkpoint with val loss: 0.13731904205264925
-- epoch 2


100%|██████████| 79/79 [00:06<00:00, 13.16it/s]


new checkpoint with val loss: 0.11642149156784709
exporting model


# export

In [ ]:
# if not os.path.exists('output/{}/backbone'.format(model_label)):
#     os.makedirs('output/{}/backbone'.format(model_label))

# where = 'output/{}'.format(model_label)
# togo = 'output/{}/backbone'.format(model_label)

# for file in os.listdir(where):
#     if file == 'backbone':
#         continue
#     os.rename(os.path.join(where, file), os.path.join(togo, file))